# Cortex Game Python Demo-Round 1

Import SASpy and connect to your SAS studio session


(Instruction of SASpy install and configuaration:
https://support.sas.com/ondemand/saspy.html )

In [ ]:
from saspy import SASsession
sas_session = SASsession()
sas_session

Load raw datasets from SAS Studio (Change the folder path to where your "Cortex Data Sets" folder located.)

In [4]:
%%SAS sas_session

libname cortex '/home/u58717790/my_shared_file_links/u39842936/Cortex Data Sets';

# Step1-2 Merge and DataPartition

link for reference: 

Pandas library: https://pandas.pydata.org/docs/user_guide/index.html


sklearn.model_selection: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [5]:
import pandas as pd

#comment: Transform cloud sas dataset to python datafrmae(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd1',
libref='cortex'
)

In [6]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge.head()
#Deal with Missing Value 
data_merge = data_merge.dropna()  #comment: maybe another method for missing data, check it later
data_merge.head()

#Step2 Data Partition 
#this is just a sample, you could use another library or bulit in funciton
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4) # you can change the percentage
train.head()

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear
523861,2523862.0,BROWN,CORNELIUS,0.0,38.0,104100.0,University / College,City,5.0,0.0,...,1.0,1.0,1.0,75.0,75.0,75.0,0.0,0.0,0.0,0.0
924123,2924124.0,DAHER,TYSON,0.0,17.0,29200.0,High School,Suburban,6.0,1.0,...,1.0,2.0,2.0,45.0,20.0,25.0,0.0,0.0,0.0,0.0
696457,2696458.0,WILLIAMS,LU,1.0,44.0,76100.0,University / College,City,6.0,2.0,...,1.0,2.0,2.0,110.0,20.0,90.0,0.0,0.0,0.0,0.0
883712,2883713.0,CLARK,CAROLYN,1.0,41.0,29900.0,University / College,City,7.0,1.0,...,1.0,2.0,4.0,160.0,40.0,120.0,1.0,20.0,1.0,10.0
77750,2077751.0,BLAHA,EMILY,1.0,37.0,63200.0,High School,Suburban,9.0,3.0,...,1.0,1.0,1.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0


# Step3 Linear Regression Model (Py)


link for reference:

sk-learn library: https://scikit-learn.org/stable/index.html

In [1]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Seniority', 'AmtLastYear']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Seniority', 'AmtLastYear']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
regr_predict=regr.predict(X_valid)

NameError: name 'train' is not defined

In [2]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

NameError: name 'Y_valid' is not defined

# Step4 Decision Tree Model（Py）

In [3]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Seniority', 'AmtLastYear']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Seniority', 'AmtLastYear']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data

NameError: name 'train' is not defined

In [19]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

19.73265743877526
11104.566135045485
105.37820521837277


### Other models may also be helpful for this game:
Link for reference:
1. Logistic regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression
2. Neural Net: https://scikit-learn.org/stable/modules/classes.html?highlight=neural%20net#module-sklearn.neural_network  

# Step6 Scoring the data

Pick the best model from previous steps and use it to predict for next year donation.

In [8]:
data3 = sas_session.sasdata2dataframe(
table='score_rd1',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)

In [9]:
scoring_data = pd.merge(data3, data4, on=["ID"],how="right")
scoring_data = scoring_data.dropna() 
scoring_data.head()

,ID,GaveLastYear,AmtLastYear,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift
0,2000001.0,0.0,0.0,ROMMES,RODNEY,0.0,25.0,107200.0,University / College,City,2.0,0.0,0.0,1.0,2.0,2.0,1010.0,10.0,1000.0
9,2000010.0,0.0,0.0,ALLEN,CHRISTY,1.0,36.0,99100.0,University / College,City,2.0,1.0,0.0,0.0,1.0,0.0,25.0,25.0,25.0
14,2000015.0,0.0,0.0,HOLMES,LEANN,1.0,39.0,91800.0,University / College,City,10.0,5.0,3.0,1.0,2.0,9.0,70.0,20.0,50.0
15,2000016.0,1.0,40.0,JOHNSTON,DONNA,1.0,28.0,72400.0,High School,Suburban,8.0,0.0,1.0,7.0,2.0,8.0,50.0,20.0,30.0
16,2000017.0,1.0,100.0,OLAGUE,DONNA,1.0,50.0,13000.0,High School,Suburban,0.0,0.0,0.0,0.0,1.0,0.0,20.0,20.0,20.0


In [10]:
X = scoring_data[['Age','Salary','Seniority','AmtLastYear']] 
regr_predict_end=regr.predict(X)
scoring_data['Prediction'] = regr_predict_end

In [16]:
scoring_data.sort_values(by=['Prediction'], inplace=True,ascending=False)
scoring_data.head()

,ID,GaveLastYear,AmtLastYear,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,Prediction
954313,2954314.0,1.0,10000.0,SANCHEZ,JADA,1.0,37.0,222700.0,High School,Suburban,10.0,2.0,1.0,1.0,4.0,9.0,95.0,10.0,40.0,295.730501
631673,2631674.0,1.0,10000.0,KOPPENHEFFER,JENNIFER,1.0,34.0,186500.0,University / College,City,9.0,3.0,1.0,0.0,1.0,0.0,500.0,500.0,500.0,291.569270
94130,2094131.0,1.0,10000.0,HOLDEN,SARAH,1.0,62.0,210000.0,University / College,Downtown,10.0,1.0,3.0,0.0,3.0,6.0,50.0,10.0,25.0,291.565983
537220,2537221.0,1.0,10000.0,BROWN,JEFFREY,0.0,48.0,212700.0,University / College,Suburban,0.0,0.0,0.0,0.0,1.0,0.0,100.0,100.0,100.0,291.464897
334249,2334250.0,1.0,10000.0,MANLEY,COLLEEN,1.0,36.0,108300.0,University / College,Suburban,6.0,4.0,3.0,5.0,2.0,6.0,45.0,20.0,25.0,288.979530


# Step7 Exporting CSV (Py)

In [18]:
Result= scoring_data[['ID','Prediction']]
Result.to_csv('Round1_Output.csv', index=False)